In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
%matplotlib inline

plt.rcParams['font.size'] = 14

# Project: Calibrating an Optical Tweezer

An [optical tweezer](https://en.wikipedia.org/wiki/Optical_tweezers) is essentially a tightly focused laser-beam, where the intensity gradient in three-dimensions tends to confine dielectric particles (usually with radii of order $100~\rm{nm}$ to $30~\mu\rm{m}$) at the location of highest intensity, i.e. the central spot of the focus. When a dielectric particle is trapped within the optical tweezer, it acts like opto-mechanical system that, for certain conditions, is well-approximated by a mass-spring system. If some external force pushes or pulls on the dielectric particle, it gets displaced from its equilibrium position, which deflects some of the optical tweezer's light and results in an optical restoring force. The restoring force is proportional to the driving force for sufficiently small displacements of the particle, i.e. $F_{\rm opt} = -k_{\rm trap} x$, where $x$ the is position of the particle along a single axis, relative to the equilibrium position. The system can thus be modeled as a **driven damped harmonic oscillator**, like the RLC circuit we've studied in Physics 81.

<img src="figures/optical_trap.png" width=510>

Making use of this property, it is possible to construct a force sensor with sensitivity to very miniscule forces, and perform precision physics experiments. Like all good appratuses, this one needs to be calibrated. This can be accomplished by pushing on the trapped particle with a known force at some fixed frequency, and then observing the response at that same frequency. We use an oscillating driving force because measurements are often difficult at DC. If we expect the system to have a different response at different frequencies, then we can drive it at multiple frequencies and observe the response at each frequency independently. This procedure measures the **transfer function** of the mechanical system, which is related to it's **mechanical susceptibility**.

To properly analyze such a calibration measurement, we'll need to apply the principles of Fourier Analysis, and compare two different signals: the driving signal and the response signal.

### The apparatus and the measurement

The experiment and the data involved in this project are based on an earlier version of the apparatus discussed in [this research paper](https://pubs.aip.org/aip/rsi/article/91/8/083201/989394/High-sensitivity-levitated-microsphere-apparatus). Essentially, an infrared laser is brought to a very tight focus inside a vacuum chamber, and silica microspheres of radii $5~\mu\rm{m}$ to $15~\mu\rm{m}$ are trapped at this focus. The microsphere and optical trap are surrounded by 6 pyramidal electrodes that allow the experimenter to drive an electric field with any direction and a wide range of magnitudes. The microsphere's motion is monitored by an optical imaging system that produces some voltage that is proportional to the displacement of the microsphere from equilibrium. Before calibration, this signal is in arbitrary units as a result of signficant digital signal processing, but we'd like to know the actual displacement/force, so we need a calibration factor from (Arbitrary units) -> (Newtons).

As part of the trapping process, the microspheres end up electrically charged. This charge can be monitored and controlled with single electron precision, and brought to a state of *exact* electrically neutrality (equal numbers of protons and electrons out of a total of $\sim 10^{13}$). This measurement is detailed in [this other research paper](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.113.251801). To calibrate the apparatus, an electric field with known magnitude is applied to a microsphere with a single electron charge, yielding a known force via the expression $F = q E$, where $q = -e$ is the charge of a single electron. The detector response (in some arbitrary units) can then be calibrated to Newtons by comparing the driving force to the measured response.

Once calibrated, the optically levitated force sensor can be used to test the neutrality of matter, or test the universal law of gravitation at short-range, where various Beyond the Standard Model (BSM) theories may manifest as short-range forces that couple to mass. We won't approach that complex of a problem in this project, rather, we'll work with some calibration data to understand the response of the force sensor.

We've provided you with a few different time-series datafiles, all with a sampling rate of $f_{\rm samp} = 5000~\rm{Hz}$.
1. `opt_lev_drive.npy`: a 10-s long data file with a single sinusoidal drive, with data in units of Volts/meter (electric field)
2. `opt_lev_response.npy`: a 10-s long data file with the response of the microsphere, with data in arbitrary detector units
3. `opt_lev_comb_drive.npy`: a 50-s long data file with a frequency comb drive (i.e. many superposed sinusoids), again in V/m
4. `opt_lev_comb_response.npy`: a 50s long data file with the resposne of the microsphere to the frequency comb, again in arbitrary detector units

This data is all taken for a single cartesian axis of drive and response, although the microsphere can be driven and measured fully in three dimensions.

The response data files include the microsphere's response to stochastic driving forces from the environment (mainly intensity and pointing fluctuations in the trapping laser) so we will have to be aware of this in our analysis. To see the single-frequency drive and response by themselves, we'll have to filter our data in the time-domain to isolate the signal we're interested in. Fourier analysis will allow us to calibrate the data *without* explicitly filtering it.

### Potential goals for this project

We expect you to complete the first two of the following goals for full credit. If you prefer to replace on of these goals with your own goal, please consult the instructor/TA.

1. Extract the amplitude and phase of the single-frequency response, relative to the drive. If we model our drive as a single tone, $f_{\rm drive} = A_{\rm drive} \cos( \omega t )$, then a general solution for a driven damped harmonic oscillator is given by $f_{\rm resp} = A_{\rm resp} \cos( \omega t + \phi)$.
    * If we take the Fourier transform of a signal, like with `np.fft.rfft`, the value of the transform at a particular frequency is generally a complex number, i.e. $\tilde{f}(\omega) = F[f(t)]$ with $F[\dots]$ the Fourier transform and with $\tilde{f}$ being complex
    * This is because most implementations of an fast fourier transform (FFT) make use of complex phasors of the form $\tilde{A}(t) = \tilde{A} e^{-i \omega t} = A e^{-i (\omega t + \phi)}$
    * The amplitude and phase of the response, relative to the drive, can be evaluated in the frequency domain by taking the absolute value of the ratio of fourier transforms and the arg of the same ratio (where 'arg' is complex-plane equivalent to the arctangent function and corresponds to the phasor rotation angle between drive and response)
    $$ A(\omega) = \frac{A_{\rm resp}}{A_{\rm drive}} = \left| \frac{\tilde{f}_{\rm resp}(\omega)}{\tilde{f}_{\rm drive}(\omega)} \right| \qquad \text{and} \qquad \phi(\omega) = \phi_{\rm resp} - \phi_{\rm drive} = \text{Arg} \left( \frac{\tilde{f}_{\rm resp}(\omega)}{\tilde{f}_{\rm drive}(\omega)} \right) $$

2. Extend this analysis to a frequency-comb drive/response and thus qualitatively determine the functions $A(\omega)$ and $\phi(\omega)$
    * The frequency comb calibration allows us to measure many different drive frequencies simultaneously.
    * Combining the amplitude ratios and phase differences for every drive frequency, we can find $A(\omega)$ and $\phi(\omega)$

3. (optional if interested) Use multiple calibration measurements and combine them to compute a mean value of the calibration factor and some standard deviation, corresponding to our certainty of the calibration factor, given the fluctuations present in the system.

### Loading the data

We can load the data into memory using NumPy. Note that the datafiles were stored as `.npy` files, which is essentially a binary file that takes advantage of compression to decrease file size (as opposed to a `.txt` file)

In [ ]:
## Sampling frequency, common to all datafiles
fsamp = 5000.0

drive = np.load('../data/opt_lev_drive.npy')
response = np.load('../data/opt_lev_response.npy')

## Make an associated time vector for the drive/response
nsamp = len(drive)
tvec = np.arange(nsamp) / fsamp

comb_drive = np.load('../data/opt_lev_comb_drive.npy')
comb_response = np.load('../data/opt_lev_comb_response.npy')
nsamp_comb = len(comb_drive)

## Make an associated time vector for the frequency comb drive/response
tvec_comb = np.arange(nsamp_comb) / fsamp

### Plotting the time-series

It's always good practice to actually look at your raw data before doing any complicated analysis. In truth, the data we've provided to you has had some pre-processing done on it to make it more palatable, but it's essentially the raw drive signal and raw microsphere response.

The drive frequency is sufficiently high, that it will be hard to see individual cycles of the driving sinusoid if we plot all 10-seconds of data at the same time, so we'll plot a subset of data: the first half-second.

In [ ]:
## Create a mask to only look at a subset of the data. The default value is 
## the first 0.5 seconds of data, but you should change this and see what happens.
mask = tvec < 0.5

fig, ax = plt.subplots(2, 1, figsize=(10, 6))
ax[0].plot(tvec[mask], drive[mask])
ax[1].plot(tvec[mask], response[mask])

ax[0].set_ylabel('E-field Drive [V/m]')
ax[1].set_ylabel('Detector Response [arb]')
ax[1].set_xlabel('Time [s]')

fig.tight_layout()
plt.show

#### Things to think about:

##### 1.1 Does the drive signal look like a single frequency sinusoid?

##### 1.2 What about the microsphere response? Can you discern anything looking at just the time-domain?

### Filtering the data

At first glance it's hard to see what's going on in the time-domain. Let's take a fourier transform of our drive signal, robustly identify the drive frequency, and then filter our microsphere response around that frequency. In this way, we can verify our assumption that the response has the form $f_{\rm resp} (t) = A_{\rm resp} \cos(\omega t + \phi)$, given a drive $f_{\rm drive} (t) = A_{\rm drive} \cos(\omega t)$

In [ ]:
## Calculate the fourier transform of the drive signal
## Note that the frequencies we calculate are in units of Hz, whereas
## 'omega' in the definitions above are in units of rad/s, with the 
## conversion factor omega = 2 * pi * freq
freqs = np.fft.rfftfreq(nsamp, 1/fsamp)
drive_fft = np.fft.rfft(drive)

## Plot the spectrum of the drive signal
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(freqs, np.abs(drive_fft))

ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Drive Spectrum [arb]')

fig.tight_layout()
plt.show()

Indeed, the Fourier transform of the drive signal looks like a single frequency together with some noise at a much lower amplitude. The single frequency looks like a delta-function at one frequency, while the rest of the spectrum is, more or less, flat across all frequencies.

By rough estimate, it looks like our drive signal is $\sim 40~\text{Hz}$. 

Can you come up with a way to find this value more precisely than just reading it off the plot?

Our estimate should be good enough to build a filter and filter the microsphere response.

In [ ]:
## Build a bandpass filter to isolate the drive frequency of ~40 Hz. We'll use a
## 4th order Butterworth filter for this, which has a pretty sharp rolloff.
sos = signal.butter(4, [30, 50], btype='band', fs=fsamp, output='sos')

## Apply the filter to the response signal, using the signal.sosfiltfilt function
## which applies the filter twice (once forward in time, once backward in time)
## to eliminate phase distortion in the output.
response_filt = signal.sosfiltfilt(sos, response)

What does our filtered signal look like in the time-domain?

Here, we'll compare it to the drive signal to see if we can see the sinusoid of the response.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(tvec[mask], drive[mask], label='Drive Signal')
ax.plot(tvec[mask], response_filt[mask]*4, label='Filtered Response (scaled for clarity)')

ax.set_xlabel('Time [s]')
ax.set_ylabel('Detector Response [arb]')
ax.legend(loc='upper right', framealpha=1, fontsize=12)

fig.tight_layout()
plt.show()

#### Things to think about:

##### 2.1 Does this look like what you expected?

It turns out that this measurement has a lot of noise and some transient features, so even after filtering the data, noise at frequencies near to the drive signal make it through our bandpass filter and appear in superposition with our response signal.

The presence of contaminating noise is somewhat expected: in this measurment, we're detecting the quasi-electrostatic response of a laboratory scale electric field (i.e. not that big) acting on a **single** electron, the smallest quanta of free-charge detectable, out of $\sim 10^{13}$ total charge carriers (both positive and negative) present on the microsphere. It's actually quite remarkable we can see a response in the first place!

##### 2.2 What does the sum of two sinusoids closely spaced in frequency look like in the time-domain? Do you see a 'beat-frequency'?

This isn't particularly important to our calibration, but helps us to make sense of what we're seeing

### Examining the Fourier Transform of the Response

The above plot begs the question: what does the Fourier transform of our signal actually look like?

In [ ]:
## Compute the FFT of the unfiltered response signal.
response_fft = np.fft.rfft(response)

fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(freqs, np.abs(response_fft))

ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Drive Spectrum [arb]')

fig.tight_layout()
plt.show()

#### Clearly there's a lot more going on in the response than just the single frequency!

There's multiple other 'tones' (single frequencies that look kind of like delta functions) in the response, as well as a lot of low-frequency noise, and some broad-spectrum features. For the calibration, we're only interested in the response at $\sim 40~\text{Hz}$.

### Suggestions moving forward

1. Plot the filtered spectrum of the response
    * Change the filter and see how this affects our response.

2. Develop a way to programatically (i.e. without just guessing based on the plot) determine the drive frequency, and then select the Fourier component of drive and response at this determined drive frequency.
    * Each element in the array `response_fft` is one such component, complex-valued, and corresponds to $\tilde{A} = A e^{- i \phi}$
    * The same index in the array `freqs` gives you $f = \omega /  2 \pi$, the frequency of that paticular Fourier component

3. Compute the amplitude ratio and phase difference using the formulas provided before
    * The `np.angle()` function will be useful here.

4. Make some similar plots of the frequency comb drive and response in both time and frequency domains

5. Extend your analysis to many freqencies simultaneously to analyze the entire frequency comb.
    * Some clever array-masking will be useful, like we did to select times less than 0.5 seconds for plotting

6. Make plots of $A(\omega)$ and $\phi(\omega)$, with appropriate axes labels